In [1]:
#install Beautiful Soup, Requests and Folium 
!pip install BeautifulSoup4
!pip install requests
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

# Import Toronto Postal Code Table from Wikipedia

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')

# Create dataframe using 'hints' from assignment.  

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    # Make sure Postal Codes that aren't assigned are not in dataframe
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# Validate Postal Codes are aligned to proper Boroughs and Neighborhoods

In [5]:
# use postal codes from assignment
sample_PCs = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
df.loc[df['PostalCode'].isin(sample_PCs)]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


# Get Longtitude and Latitude data for Toronto Postal codes from online csv file and merge into table with PostalCode, Borough and Neighborhood data

In [6]:
#Create dataframe of Postal Codes Lattitudes and Longtitudes from online file
GEO_URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
coordinates = pd.read_csv(GEO_URL)
#Merge above dataframe with PostalCode, Borough and Neighborhood dataframe
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df = df.join(coordinates.set_index('PostalCode'), on='PostalCode')

# Validate Latitude and Longtitude data is aligned with Borough and Neighborhood data 

In [7]:
#use postal codes from assignment
sample_PCs = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
df.loc[df['PostalCode'].isin(sample_PCs)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
23,M4G,East York,Leaside,43.709060,-79.363452
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
50,M9L,North York,Humber Summit,43.756303,-79.565963
54,M4M,East Toronto,Studio District,43.659526,-79.340923
71,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849


# Explore and cluster the neighborhoods in Toronto.

In [8]:
# Get the geo coordinates of Toronto from geopy
geolocator = Nominatim(user_agent="Paul_Bristow")
location = geolocator.geocode('toronto')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create a Map of Toronto with markers for ALL Postal Codes, Boroughs and Neighborhoods in extracted dataframe

In [9]:
# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map using Postal Code, Boroughs and Neighborhoods
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#99ee90',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Create a Map of Toronto with markers for only those Postal Codes and Neighborhoods where the boroughs contain the name "Toronto" within them.

In [10]:
TorontoBoroughs = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
print("There are ", TorontoBoroughs.shape[0], "Boroughs that contain 'Toronto' in the city of Toronto")

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoBoroughs['Latitude'], TorontoBoroughs['Longitude'], TorontoBoroughs['Borough'], TorontoBoroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#99ee90',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


There are  39 Boroughs that contain 'Toronto' in the city of Toronto


In [12]:
CLIENT_ID = 'YASCKOPZRKN5LFUNFOH32LUQWLYI5TQYTQAVZIDEXKTWIU03' # your Foursquare ID
CLIENT_SECRET = 'L2VBXNE40CM4PZFNEA334COCDTUIZT4QRNU5BEQZJPKKOORH' # your Foursquare Secret
ACCESS_TOKEN = 'CIU5T2CSEI0AMGPWBALZQX5QEK30QWECTMVRVGPBVJO2YNF5' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

In [13]:
TorontoBoroughs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


# Get the venues by boroughs and neighborhood for those Postal Codes with Boroughs that contain 'Toronto'

In [14]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
borough_venues = getNearbyVenues(boroughs=TorontoBoroughs['Borough'],
                                             neighborhoods=TorontoBoroughs['Neighborhood'],
                                             latitudes=TorontoBoroughs['Latitude'],
                                             longitudes=TorontoBoroughs['Longitude'])
borough_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


## Show Number of Venues grouped by Borough and Neighborhood

In [16]:
print("There are ", borough_venues.shape[0], " in those ", TorontoBoroughs.shape[0], "bouroughs that have 'Toronto' within their names")

There are  841  in those  39 bouroughs that have 'Toronto' within their names


In [17]:
borough_venues.groupby(['Borough', 'Neighborhood']).count()

Neighborhood Latitude  \
Borough                Neighborhood                                                                
Central Toronto        Davisville                                                             30   
                       Davisville North                                                       10   
                       Forest Hill North & West                                                4   
                       Lawrence Park                                                           4   
                       Moore Park, Summerhill East                                             3   
                       North Toronto West                                                     20   
                       Roselawn                                                                4   
                       Summerhill West, Rathnelly, South Hill, Forest ...                     14   
                       The Annex, North Midtown, Yorkville                                    21   
Downtown Toronto       Berczy Park                                                            30   
                       CN Tower, King and Spadina, Railway Lands, Harb...                     18   
                       Central Bay Street                                                     30   
                       Christie                                                               16   
                       Church and Wellesley                                                   30   
                       Commerce Court, Victoria Hotel                                         30   
                       First Canadian Place, Underground city                                 30   
                       Garden District, Ryerson                                               30   
                       Harbourfront East, Union Station, Toronto Islands                      30   
                       Kensington Market, Chinatown, Grange Park                              30   
                       Regent Park, Harbourfront                                              30   
                       Richmond, Adelaide, King                                               30   
                       Rosedale                                                                4   
                       St. James Town                                                         30   
                       St. James Town, Cabbagetown                                            30   
                       Toronto Dominion Centre, Design Exchange                               30   
                       University of Toronto, Harbord                                         30   
Downtown Toronto Stn A Enclave of M5E                                                         30   
East Toronto           India Bazaar, The Beaches West                                         22   
                       Studio District                                                        30   
                       The Beaches                                                             4   
                       The Danforth West, Riverdale                                           30   
East Toronto Business  Enclave of M4L                                                         17   
East York/East Toronto The Danforth  East                                                      2   
West Toronto           Brockton, Parkdale Village, Exhibition Place                           24   
                       Dufferin, Dovercourt Village                                           15   
                       High Park, The Junction South                                          25   
                       Little Portugal, Trinity                                               30   
                       Parkdale, Roncesvalles                                                 14   
                       Runnymede, Swansea                                  

## Look at the number of venues catagories by Borough and Neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = borough_venues['Borough'] 
toronto_onehot['Neighborhoods'] = borough_venues['Neighborhood'] 
toronto_onehot['Venue'] = borough_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Neighborhoods,Venue,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Downtown Toronto,"Regent Park, Harbourfront",Roselle Desserts,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,"Regent Park, Harbourfront",Tandem Coffee,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,"Regent Park, Harbourfront",Cooper Koo Family YMCA,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,"Regent Park, Harbourfront",Body Blitz Spa East,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,"Regent Park, Harbourfront",Impact Kitchen,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Determine the size of the dataframe

In [20]:
toronto_onehot.shape

(841, 206)

### Group rows by borough and neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
Grouped_TO = toronto_onehot.groupby(['Borough', 'Neighborhoods']).mean().reset_index()
Grouped_TO

,Borough,Neighborhoods,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Toronto,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000
6,Central Toronto,Roselawn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000
8,Central Toronto,"The Annex, North Midtown, Yorkville",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Downtown Toronto,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
